In [1]:
import os
import sys
import pandas
import time
import csv
import json
import openpyxl
from IPython.display import display, HTML

#the bail ist the following: 1) list every IMP province ordered by desc road number until 2. (under it's a dead end and we don't care)
# 2 = minor 
global global_dupl_count
global_dupl_count=0
def dict_raise_on_duplicates(ordered_pairs):
    """Reject duplicate keys."""
    global global_dupl_count
    d = {}
    for k, v in ordered_pairs:
        if k in d:
            global_dupl_count=global_dupl_count+1
            d[k+str(global_dupl_count)] = v
        else:
            d[k] = v
    return d

#filepath = "../Imperator/save games/"
filepath = "D:/python/Imperator/01_json/mp_rome_grand_camp/"
filename = "mp_rome_grand_camp31-01-2024_00h17"
savegame = filepath+filename+".json"

with open(savegame, encoding="utf-8") as source:
    source=json.load(source,object_pairs_hook=dict_raise_on_duplicates)
#with open("imperator/mappings/countries_names.csv") as countries_names:
#    countries_names = pandas.read_csv(countries_names, sep=";", header=None)
#    countries_names.columns = ["country_name_key", "country_name"]
        

In [2]:
from imperator import country_analyses
from imperator import state_analyses
from imperator import province_analyses

from imperator import pop_analyses
from imperator import char_analyses
from imperator import rulerterm_analyses

from imperator import war_analyses
from imperator import battle_analyses
from imperator import family_analyses




countries = country_analyses.extract(source)
countries["date"]=source["date"]
states = state_analyses.extract(source)
states["date"]=source["date"]
provinces = province_analyses.extract(source)[0]
provinces["date"]=source["date"]

pops = pop_analyses.extract(source).join(province_analyses.extract(source)[1])

chars = char_analyses.extract(source)
rulerterms = rulerterm_analyses.extract(source)
families = family_analyses.extract(source)
wars = war_analyses.extract(source)
battles = battle_analyses.extract(source)

#countries["country_name_key"]=countries["historical"]
#countries["total_income"]=round(countries["economy_lastmonth_tax_income"]+countries["economy_lastmonth_tax_income"]+countries["economy_lastmonth_tax_income"])
#countries = countries.sort_values("total_holdings",ascending=False).query('total_income > 10 & total_holdings > 0')

#with pandas.ExcelWriter("D:/python/"+filename+".xlsx") as writer:

    #rulerterms.to_excel(writer, sheet_name='rulers')  
    #countries.to_excel(writer, sheet_name='countries')  
    #provinces.to_excel(writer, sheet_name='provinces')  
    #pops.to_excel(writer, sheet_name='pops')  
    #chars.to_excel(writer, sheet_name='chars')  
    #wars.to_excel(writer, sheet_name='wars')
    #battles.to_excel(writer, sheet_name='battles')

#freeze_header(countries[countries.total_population!=0].sort_values(by=["total_population"],ascending=False))
# freeze_header(provinces.sort_values(by=["pop_count"],ascending=False))


In [3]:
source.keys()
# source["country"]['country_database']["112"]['governorship'].keys()
list = []
j=0
# newlist = [x for x in source["country"]['country_database']["112"] if "governorship" in x]
for i in source["country"]['country_database']["112"].keys():
    # list.append(source["country"]['country_database']["112"][i])

    if "governorship" in i:
        # j+=1
        # list.append(i)
        # list.append(source["country"]['country_database']["112"][i]["region"])
        # a=1
        for j in source["country"]['country_database']["112"][i]:
            if "governor" in j:
                governor = "Governor "+chars.at[str(source["country"]['country_database']["112"][i][j]),"first_name"]+" "+chars.at[str(source["country"]['country_database']["112"][i][j]),"family_name"]
            elif "legion" in j:
                legion = source["country"]['country_database']["112"][i][j]
                # list.append([source["country"]['country_database']["112"][i]["region"],])
            elif "disband_date" in j:
                disband_date = source["country"]['country_database']["112"][i][j]
            elif not("cached") in j and not("debug") in j:
                list.append([source["country"]['country_database']["112"][i]["region"],j])
                    # for k in source["country"]['country_database']["112"][i][j]["list"]:
                        # print (k)
                        # if not("to_sell") in source["country"]['country_database']["112"][i][j]["list"][k]["name"]:
                            # list.append([source["country"]['country_database']["112"][i]["qregion"],source["country"]['country_database']["112"][i][j][k]])
                        # a=1
# source["country"]['country_database']["112"]['governorship'].keys()
# list

file="C:/Users/PH/DLApps/ImperatorToCK3-win-x64/ImperatorToCK3/configurables/province_mappings.txt"
province_mapping = open(file, encoding="latin-1")
level = 0
imp_prov=0
investigate=[]
prov=0
road_imp=[]
prov_ck3=[]
for line in province_mapping:
    x = line.strip()
    if "{" in x: level+=1
    if "}" in x: level-=1

    if "imperator_invictus" in x: is_imperator_invictus=1
    elif is_imperator_invictus==1 and level==0: is_imperator_invictus=0

    if "imp =" in x and is_imperator_invictus==1:
        prov+=1
        count_imp=0
        count_ck=0
        roads=0
        cur_ck=[]
        name=""
        #loop imp in line
        first_imp=x.find("imp =")
        for i in range(x.count("imp =")):
            next_imp=x.find("imp =",first_imp+1)
            if next_imp<=0:next_imp=x.find("ck3 =")
            left=x[first_imp:next_imp]
            left_imp=left[left.find("=")+1:]
            try: roads += int(provinces.at[str(left_imp.strip()),"num_of_roads"])
                
            except:roads = 0
            try: name = provinces.at[str(left_imp.strip()),"province_name_text"]+" ("+str(left_imp.strip())+")"
            except:roads = 0

            investigate.append([prov,"imp",left_imp,roads])
            road_imp.append([left_imp,roads]) 
            first_imp = next_imp
            count_imp+=1
        #loop ck3 in line
        roads=roads/count_imp
        first_ck3=x.find("ck3 =")
        for i in range(x.count("ck3 =")):
            next_ck3=x.find("ck3 =",first_ck3+1)
            # if next_imp<=0:next_imp=x.find("ck3 =")
            left=x[first_ck3:next_ck3]
            left_ck3=left[left.find("=")+1:]
            if "}" in left_ck3:left_ck3=left_ck3[:left_ck3.find("}")]
            
            cur_ck.append([prov,left_ck3])
            first_ck3 = next_ck3
            count_ck+=1
        
        # if left_ck3 in road_imp:
        if count_ck>0: road_per_ck3=roads/count_ck
        
        for ckprovs in cur_ck:
            if road_per_ck3>0:
                # print(ckprovs[1],road_per_ck3)
                prov_ck3.append([ckprovs[1],round(road_per_ck3,0),name,roads,count_ck])

        
        # test_mapping.append([investigate[prov][]])

# debug = pandas.DataFrame(investigate)
# debug.columns =["prov_mapping","prov_game","prov_id","roads"]
                
debug = pandas.DataFrame(prov_ck3)
debug.columns =["ckprovs[1]","round(road_per_ck3,0)","name","roads","count_ck"]

# debug = pandas.DataFrame(prov_ck3)
# debug.columns =["prov_mapping","prov_game","prov_id","roads"]

# for ck in 
# with pandas.ExcelWriter("D:/python/"+filename+"_roads.xlsx") as writer:

    # debug.to_excel(writer, sheet_name='roads')  


PROV3
